In [355]:
import pandas as pd
import os
import random
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

## Intro
- 앞서 `get_feautre.ipynb` 에서 구한 파일들을이용해 실제로 valid/ test set을 분리 해보겠습니다.

In [356]:
path = "/opt/ml/input/data"

# train_data 를 불러올 때는 dtype을 명시해줘서 불러옵니다

train_df = pd.read_csv(os.path.join(path,'train_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)


### add_col
- 'cum_correct'라는 새로운 열을 추가해줍니다.
- 해당 문제를 풀기 전까지 이 `testId`(시험지) 에서의 정답률을 나타냅니다.

In [357]:
def add_col(df):
    pre = df["testId"][0]
    count = df["answerCode"][0]
    c = 1
    new = []

    for idx, answer in zip(df["testId"],df["answerCode"]):
        if pre != idx :
            pre = idx
            new.append(0)
            c = 1
            count = answer
        else :
            new.append(count/c)
            c += 1
            count += answer
    df['cum_correct'] = new
    return df

### add_list_problem
- 해당 유저가 푸는 시험지의 마지막 문제에 -1 로 마킹을 해줍니다.

In [358]:
def add_last_problem(df):
    new = []
    pre = df['testId'][0]
    for idx in df['testId']:
        if pre != idx :
            new[-1]=-1
            pre = idx
        new.append(0)
    df['last_problem'] = new
    return df

### feature_engineering
- 유저의 문제 풀이 숫자, 정답 숫저, 정답률을 시간순으로 누적해줍니다.
- 문제 풀이이 시간의 이상치(650 이상)을 제거해줍니다.

In [359]:
def feature_engineering(df, testId_acc, knowLedgedTag_acc, userID_feature, testId_feature):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    df = add_col(df)
    df = add_last_problem(df)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']
    

    # diff()를 이용하여 시간 차이를 구해줍니다
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    
    # 만약 0초만에 풀었으면 0으로 치환
    diff = diff.fillna(pd.Timedelta(seconds=0))
    
    # 시간을 전부 초단위로 변경합니다.
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

    # df에 elapsed(문제 풀이 시간)을 추가해줍니다.
    df['elapsed'] = diff
    
    # 문제 풀이 시간이 650초 이상은 이상치로 판단하고 제거합니다.
    df['elapsed'] = df['elapsed'].apply(lambda x : x if x <650 else None)
    
    # 대분류(앞 세자리)
    df['grade']=df['testId'].apply(lambda x : int(x[1:4])//10)

    # 중분류(중간 세자리)
    df['mid'] = df['testId'].apply(lambda x : int(x[-3:]))

    # 문제 번호(분류를 제외한)
    df['problem_number'] = df['assessmentItemID'].apply(lambda x : int(x[-3:]))
    
    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산 합니다.
    
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용 합니다.
    df = pd.merge(df, testId_acc, on=['testId'], how="left")
    df = pd.merge(df, knowLedgedTag_acc, on=['KnowledgeTag'], how="left")
    df = pd.merge(df, userID_feature, on=['userID','grade'], how="left")
    df = pd.merge(df, testId_feature, on=['testId'], how="left")
    
    df['problem_position'] = df['problem_number'] / df["problem_count"]
    
    return df

In [360]:
# testId(시험지)의 평균 정답률을 구합니다.(시헙지별 난이도)
testId_acc = train_df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
testId_acc.columns = ["test_mean", 'test_sum']

# 마찬가지로 KnowledgedTag의 평균 정답률을 구하여 대략적인 난이도를 구합니다
knowLedgedTag_acc = train_df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
knowLedgedTag_acc.columns = ["tag_mean", 'tag_sum']

userID_feature = pd.read_csv(os.path.join(path, "feature/userID_feature.csv"), index_col = False)
testId_feature = pd.read_csv(os.path.join(path,"feature/testId_feature.csv"),index_col = False)

In [361]:
# 전부 붙여줍니다.
train_df = feature_engineering(train_df, testId_acc, knowLedgedTag_acc, userID_feature, testId_feature)
train_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,cum_correct,last_problem,user_correct_answer,user_total_answer,...,Unnamed: 0_x,grade_acc,grade_count,grade_elapsed,Unnamed: 0_y,problem_count,type_count,test_acc,test_elapsed,problem_position
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1.0,0,NaN,0,...,1,0.791908,346,36.533784,975,7,2,0.952667,20.842105,0.142857
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,0,1.0,1,...,1,0.791908,346,36.533784,975,7,2,0.952667,20.842105,0.285714
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1.0,0,2.0,2,...,1,0.791908,346,36.533784,975,7,2,0.952667,20.842105,0.428571
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1.0,0,3.0,3,...,1,0.791908,346,36.533784,975,7,2,0.952667,20.842105,0.571429
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1.0,0,4.0,4,...,1,0.791908,346,36.533784,975,7,2,0.952667,20.842105,0.714286


### test_data도 똑같이 처리해줍니다

In [362]:
test_df = pd.read_csv(os.path.join(path,'test_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [363]:
# FEATURE ENGINEERING
test_df = feature_engineering(test_df, testId_acc, knowLedgedTag_acc, userID_feature, testId_feature)
test_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,cum_correct,last_problem,user_correct_answer,user_total_answer,...,Unnamed: 0_x,grade_acc,grade_count,grade_elapsed,Unnamed: 0_y,problem_count,type_count,test_acc,test_elapsed,problem_position
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,1.00,0,NaN,0,...,8,0.655052,861,63.258152,804,7,4,0.549714,68.609658,0.142857
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,1.00,0,1.0,1,...,8,0.655052,861,63.258152,804,7,4,0.549714,68.609658,0.285714
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,1.00,0,2.0,2,...,8,0.655052,861,63.258152,804,7,4,0.549714,68.609658,0.428571
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.75,0,2.0,3,...,8,0.655052,861,63.258152,804,7,4,0.549714,68.609658,0.571429
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.60,0,2.0,4,...,8,0.655052,861,63.258152,804,7,4,0.549714,68.609658,0.857143


In [364]:
test_df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'cum_correct', 'last_problem', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'elapsed', 'grade', 'mid',
       'problem_number', 'test_mean', 'test_sum', 'tag_mean', 'tag_sum',
       'Unnamed: 0_x', 'grade_acc', 'grade_count', 'grade_elapsed',
       'Unnamed: 0_y', 'problem_count', 'type_count', 'test_acc',
       'test_elapsed', 'problem_position'],
      dtype='object')

In [327]:
blank = test_df[test_df['answerCode']==-1].copy()

In [328]:
len(blank['userID'])

744

### DATA SPLIT
- 이제 데이터를 나눠줍니다.

In [365]:
from collections import defaultdict


In [366]:
def get_grade_mid(x):
    dic = defaultdict(set)
    for i in x:
        grade = int(i[1:4])//10
        mid = int(i[-3:]) // 10
        dic[grade].add(mid)
    return dic

### 비슷한 User?
- 정답률이 비슷하고
- 풀이시간이 비슷하면
- 학습정도가 유사한(비슷한 유저)라고 볼 수 있지 않을까요?

### 비슷한 학습 경험
- seq length가 유사하고(풀이 경험)
- testId가 비슷하면 비슷한 User가 아닐가요?
- ex) {3: {7}, 4: {16}} 라고 되어 있으면 3번 대분류의 7번 시험지, 4번 대분류의 16번 시험지를 푼 유저입니다.

In [367]:
# 마지막 index추출해주는 lambda 생성
# 마지막 index를 가져오면 최근까지의 누적 학습 정답률과 풀이 갯수를 가저올 수 있습니다
current = lambda x : list(x)[-1]

TRAINID = train_df.groupby(['userID']).agg({'testId':get_grade_mid,'answerCode':'count','elapsed':'mean','grade_acc':current, 'grade_count':current, 'grade':current, 'mid':current}).reset_index()

TRAINID.columns = ['userID', 'testId', 'solve_count', 'elapsed', 'grade_acc','current_grade_count', 'current_grade', 'current_mid']
TRAINID = TRAINID.sort_values(by=['solve_count', 'current_grade','current_mid']).reset_index(drop=True)
TRAINID[:10]

,userID,testId,solve_count,elapsed,grade_acc,current_grade_count,current_grade,current_mid
0,7441,"{3: {7}, 4: {16}}",9,43.750000,1.000000,4,4,165
1,7414,"{4: {19}, 3: {2}, 2: {12}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {1}, 5: {19}, 4: {13}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {7}, 5: {19}}",13,22.454545,1.000000,4,5,193
4,7386,"{1: {1, 15}, 4: {13}}",14,28.000000,0.600000,10,1,155
5,6981,"{2: {19, 14}, 4: {19}}",14,26.666667,0.888889,9,2,143
6,7189,"{3: {14, 7}, 2: {17}}",14,36.833333,0.250000,4,2,179
7,7330,"{4: {9}, 1: {14}, 2: {17}}",14,22.333333,1.000000,4,2,179
8,7349,"{6: {18}, 4: {16}, 3: {12}}",14,68.500000,0.400000,5,3,126
9,7422,"{6: {9}, 3: {17, 4}}",14,20.500000,0.300000,10,3,171


In [368]:
TESTID = test_df.groupby(['userID']).agg({'testId':get_grade_mid,'answerCode':'count','elapsed':'mean','grade_acc' : current, 'grade_count' : current, 'grade' : current, 'mid' : current}).reset_index()

TESTID.columns = ['userID', 'testId', 'solve_count', 'elapsed', 'grade_acc', 'current_grade_count', 'current_grade', 'current_mid']
TESTID = TESTID.sort_values(by=['solve_count', 'current_grade','current_mid']).reset_index(drop=True)
TESTID[:10]

,userID,testId,solve_count,elapsed,grade_acc,current_grade_count,current_grade,current_mid
0,5965,"{3: {0, 18}, 2: {15}}",15,29.000000,0.500000,4,2,159
1,7404,"{2: {12}, 4: {10}, 3: {11}}",15,33.846154,0.500000,4,3,111
2,7033,"{4: {8, 18, 13}}",15,9.000000,0.214286,14,4,133
3,7354,"{1: {10}, 3: {14}, 4: {17}}",15,66.923077,0.250000,4,4,170
4,7373,"{3: {15}, 4: {18}, 5: {17}}",15,24.615385,1.000000,4,5,178
5,7416,"{8: {0}, 3: {16}, 5: {19}}",15,29.461538,0.666667,3,5,193
6,7417,"{1: {9, 10}, 5: {19}}",15,50.916667,0.666667,3,5,193
7,7216,"{2: {17, 18}, 3: {19}}",16,51.714286,0.400000,10,2,170
8,7439,"{4: {0, 19, 13}}",16,37.928571,0.733333,15,4,130
9,7146,"{1: {2}, 4: {19}, 6: {6}}",16,38.571429,0.000000,3,6,67


### Valid 추출
TRAIN에서 TEST와 비슷한 VALID를 추출하기 위해선 다음과 같은 우선순위를 가집니다.

1. 마지막으로 풀었던 문제의 대분류가 같은가?
2. 마지막 대분류에서 풀었던 문제의 갯수가 비슷한가?(상위 30개)
3. 상위 30개중에서 중분류가 똑 같은 유저가 존재하는가?
4. 없다면 가장 정답률이 비슷한 최종 1명 추출

In [370]:
tdict = {}

# 적용 기준 

for _, row in TESTID.iterrows():
    # 1. 마지막으로 풀었던 문제의 대분류가 같은가?
    candidate = TRAINID[TRAINID['current_grade']==row['current_grade']].copy()

    # 선택되지 않은 것을 골라냅니다.
    candidate = candidate[candidate['userID'].apply(lambda x : x in tdict.values()) == False]
    
    # 2. 마지막 대분류에서 풀었던 문제의 갯수가 비슷한가?(상위 30개)
    candidate['current_grade_count'] = candidate['current_grade_count'].apply(lambda x : abs(x-row['current_grade_count']))
    candidate = candidate.sort_values(by='current_grade_count').reset_index(drop=True)
    candidate = candidate[:min(candidate.shape[0],30)].copy()
    
    # 3. 상위 30개중에서 중분류가 똑 같은 유저가 존재하는가?
    # 4. 없다면 가장 정답률이 비슷한 최종 1명 추출
    candidate['r_acc'] = (candidate['grade_acc'] - row['grade_acc']).apply(lambda x :abs(x))
    candidate['r_mid'] = (candidate['current_mid'] == row['current_mid']).astype(int)
    candidate = candidate.sort_values(by=['r_mid','r_acc']).reset_index(drop=True)
    tdict[row['userID']] = list(candidate['userID'])[0]

print('done!')
print(f'please check {len(tdict)} == {len(set(tdict.values()))}')

done!
please check 744 == 744


In [371]:
# 한번 정렬해줍니다.
val_index = sorted(list(tdict.values()))

In [372]:
# Train에서 추출
validset = TRAINID[TRAINID["userID"].isin(val_index)]

In [373]:
validset

,userID,testId,solve_count,elapsed,grade_acc,current_grade_count,current_grade,current_mid
1,7414,"{4: {19}, 3: {2}, 2: {12}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {1}, 5: {19}, 4: {13}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {7}, 5: {19}}",13,22.454545,1.000000,4,5,193
5,6981,"{2: {19, 14}, 4: {19}}",14,26.666667,0.888889,9,2,143
8,7349,"{6: {18}, 4: {16}, 3: {12}}",14,68.500000,0.400000,5,3,126
...,...,...,...,...,...,...,...,...
6633,215,"{7: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1326,53.961572,0.677140,923,7,138
6634,886,"{7: {0, 1, 2, 3, 4}, 3: {0, 1, 2, 3, 4, 5, 6, ...",1329,55.566516,0.719902,407,5,131
6636,433,"{5: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1338,45.134927,0.508423,653,7,161
6667,174,"{4: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1525,58.796935,0.426891,595,7,137


### Test와 valid가 유사한지 비교해보겠습니다.
- userID_test : 원래 test 유저
- userID_valid : userID_test와 비슷한 valid 유저
- current_garde_test(valid) : test(valid) 유저가 최근에 푼 대분류의 종류
- grade_acc_test(valid) : test(valid) 유저가 최근에 푼 대분류의 문제의 평균 정답률
- current_count_test(valid) : test(valid)유저가 해당 대분류에 대해 푼 문제의 개수
- count_test(valid) : test(valid)유저가 푼 전체 문제의 개수
- current_mid_test(valid) : test(valid) 유저가 최근에 푼 시험지의 종류

In [374]:
TESTID['similar'] = TESTID['userID'].apply(lambda x : tdict[x]) 


In [375]:
compare = pd.merge(TESTID,validset,left_on='similar',right_on='userID',how='inner')


In [376]:
compare.columns =['userID_test', 'testId_test', 'solve_count_test', 'elapsed_test', 'grade_acc_test',
       'current_count_test', 'current_grade_test', 'current_mid_test', 'similar', 'userID_valid',
       'testId_valid', 'solve_count_valid', 'elapsed_valid', 'grade_acc_valid', 'current_count_valid',
       'current_grade_valid', 'current_mid_valid']

### 보는 방법
- ex) 3번 test 유저와 92번 valid 유저는 비슷합니다.
    - 같은 대분류 5번을 최근에 풀었으며
    - 0.6550, 0.6530으로 정답률이 비슷합니다.
    - 최근 까지 푼 대분류 문제의 개수는 861, 588개로 비슷하지 않습니다.
    - 하지만 풀었던 모든 문제의 개수는 1036개, 973개로 비슷합니다.

In [420]:
compare[['userID_test', 'userID_valid','current_grade_test','current_grade_valid','grade_acc_test',
         'grade_acc_valid','current_count_test','current_count_valid',
         'solve_count_test','solve_count_valid','current_mid_test','current_mid_valid']].sort_values(by=['userID_test']).head(30)


,userID_test,userID_valid,current_grade_test,current_grade_valid,grade_acc_test,grade_acc_valid,current_count_test,current_count_valid,solve_count_test,solve_count_valid,current_mid_test,current_mid_valid
717,3,92,5,5,0.655052,0.653061,861,588,1036,973,133,155
610,4,4045,7,7,0.703529,0.678218,425,404,671,886,146,134
738,13,868,7,7,0.390593,0.385417,489,480,1317,670,111,157
734,17,2434,9,9,0.924757,0.921986,412,423,1260,810,64,67
468,26,3636,6,6,0.817365,0.820598,334,301,387,349,135,134
675,29,926,2,2,0.891667,0.886275,480,510,854,1773,190,169
723,45,2013,4,4,0.556075,0.556054,214,223,1084,439,136,144
615,53,1615,4,4,0.511416,0.556777,657,546,693,703,140,142
662,58,1754,7,7,0.230645,0.215035,620,572,811,650,159,161
735,64,2246,7,7,0.617486,0.627072,366,362,1270,389,146,154


### 중복된 유저는 없는가?

In [418]:
compare.userID_valid.nunique() == compare.userID_test.nunique()

True

### 모든 유저들의 대분류가 같은가?

In [378]:
count = 0
for user_t, user_v in zip(compare.userID_test, compare.userID_valid):
    if set(compare.current_grade_test[compare.userID_test==user_t]) == set(compare.current_grade_valid[compare.userID_valid==user_v]):
        count+=1
if count == 744:print(True)
else:print(False)

True


### 대분류와 중분류가 같은 유저는?

In [397]:
count=0
for user_t, user_v in zip(compare.userID_test, compare.userID_valid):
    if str(list(compare.current_mid_test[compare.userID_test==user_t])[0])[:-1] == str(list(compare.current_mid_valid[compare.userID_valid==user_v])[0])[:-1]:
        count+=1
print(f'744 개중 {count}개')


744 개중 121개


### 대분류별 평균 점수의 유사도는?

In [416]:

grade_mean = 0
for user_t, user_v in zip(compare.userID_test, compare.userID_valid):
    t = list(compare.grade_acc_test[compare.userID_test==user_t])[0]
    v = list(compare.grade_acc_valid[compare.userID_valid==user_v])[0]
    if t == 0 or v==0:
        continue
    grade_mean += min(t,v)/max(t,v)

print(round(grade_mean/744,5)*100,'%')

95.044 %


### 새로운 파일 저장
3가지 파일을 저장할 것입니다.
- `cv_train_data.csv` : `test`, `valid`를 포함할지 포함지 않을지 결정해서 학습을 시킵니다.
- `cv_valid_data.csv` : 위 일련의 과정을 통해 비슷한 test데이터의 user들과 비슷한 유저를 추출한 파일입니다. 마지막으로 푼 문제에는 -1로 마킹 해줍니다.
- `valid_target.csv` : -1로 마킹하기전에 valid의 정답들을 저장한 파일입니다.

### cv_train_data.csv

In [380]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
train_df = pd.read_csv(os.path.join(path, 'train_data.csv') , dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

test_df = pd.read_csv(os.path.join(path, 'test_data.csv') , dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
test_df = test_df[test_df['answerCode']!= -1].copy()

In [381]:
# validation 정보를 train에 포함 시키지 않으려면 실행!!
cv_train_df = train_df[train_df["userID"].apply(lambda x : x not in val_index)]

# test 정보를 train에 포함 시키려면 실행!
# cv_train_df = pd.concat([train_df,test_df])
cv_train_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


In [382]:
print(train_df.userID.nunique()) # 원래 train의 user 숫자
print(cv_train_df.userID.nunique()) # valid를 포함시키면 그대로 6698, 제외하면 5954 입니다.

6698
5954


### cv_valid_df.csv

In [383]:
# train에서 val_index의 유저들을 가져옵니다.
valid_df = train_df[train_df["userID"].apply(lambda x : x in val_index)]
valid_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
26036,35,A040093001,A040000093,1,2020-01-02 23:30:13,2094
26037,35,A040093002,A040000093,0,2020-01-02 23:31:13,2094
26038,35,A040093003,A040000093,0,2020-01-02 23:31:56,2094
26039,35,A040093004,A040000093,0,2020-01-02 23:32:15,2094
26040,35,A040093005,A040000093,1,2020-01-02 23:35:06,2094


### valid_target.csv
- validation score를 구하기위해서 마지막 문제는 따로 분리 해줍니다.

In [384]:
valid_target = valid_df.groupby('userID').tail(1)
valid_target

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
26726,35,A010112006,A010000112,1,2020-10-26 10:15:39,6777
27262,36,A070148008,A070000148,0,2020-12-18 08:26:33,9083
32362,43,A050135007,A050000135,0,2020-10-26 07:49:41,5298
44485,59,A070160008,A070000160,1,2020-12-27 04:37:26,9122
51142,68,A080130008,A080000130,0,2020-12-13 09:18:17,8404
...,...,...,...,...,...,...
2265801,7388,A070121001,A070000121,0,2020-11-05 03:29:30,8910
2265831,7390,A050193004,A050000193,1,2020-09-29 08:35:29,10402
2265915,7397,A040169005,A040000169,1,2020-10-07 12:36:48,2118
2266183,7414,A020124005,A020000124,0,2020-09-10 03:25:08,8018


In [386]:
# valid정답 정보를 추출했으니 마지막 문제를 -1로 masking 처리합니다.
for idx in tqdm(valid_target.index):
    valid_df.answerCode[valid_df.index==idx] = -1

  0%|          | 0/744 [00:00<?, ?it/s]/tmp/ipykernel_124787/1733076662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.answerCode[valid_df.index==idx] = -1
100%|██████████| 744/744 [00:08<00:00, 83.48it/s]


In [387]:
# 제대로 masking되었는지 확인
valid_df.index[valid_df.answerCode==-1].size

744

In [389]:
# column 이름 변경
valid_target = valid_target[['answerCode']]
valid_target.columns=[['target']]

In [390]:
# 저장
cv_train_df.to_csv("/opt/ml/input/data/cv_train_data.csv",index=False)
valid_df.to_csv("/opt/ml/input/data/cv_valid_data.csv",index=False)
valid_target.to_csv("/opt/ml/input/data/valid_target.csv",index=False)